### Recommendations Rank & Knowledge Based: 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv(r'C:\Users\Vista\Desktop\movies_clean.csv')
reviews = pd.read_csv(r'C:\Users\Vista\Desktop\reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
reviews.shape

(712337, 23)

In [4]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [5]:
movies.shape

(31245, 35)

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [6]:
reviews.movie_id.value_counts()

1454468    3029
993846     2673
770828     2617
816692     2572
816711     2385
           ... 
23814         1
42265         1
101692        1
103743        1
62177         1
Name: movie_id, Length: 31245, dtype: int64

In [7]:
reviews.movie_id.value_counts().index[:10]

Int64Index([1454468,  993846,  770828,  816692,  816711, 1670345, 1300854,
            1535109, 2267998, 1392214],
           dtype='int64')

In [8]:
movies.query("movie_id==1454468")['movie']

17747    Gravity (2013)
Name: movie, dtype: object

In [9]:
reviews.query("movie_id==1454468").rating.value_counts()

9     852
8     819
10    646
7     376
6     179
5      70
4      31
1      22
2      21
3      13
Name: rating, dtype: int64

In [10]:
#avg
rate =reviews.query("movie_id==1454468").rating.value_counts()
m = movies.query("movie_id==1454468")['movie']

v=rate.values
i=rate.index
print(i)
print(v)

avg=sum(i*v)/sum(v)
print(f"average rating of movie: {m} is: {avg} ")


Int64Index([9, 8, 10, 7, 6, 5, 4, 1, 2, 3], dtype='int64')
[852 819 646 376 179  70  31  22  21  13]
average rating of movie: 17747    Gravity (2013)
Name: movie, dtype: object is: 8.241333773522614 


In [11]:
reviews.query("rating == 10")['movie_id'].value_counts()

816692     1142
1454468     646
111161      573
993846      543
2119532     539
           ... 
1320103       1
2821314       1
120025        1
910559        1
1077262       1
Name: movie_id, Length: 10011, dtype: int64

In [12]:
reviews.query("movie_id == 816692")['rating']

81         8
371       10
394        9
555        9
680        8
          ..
711185    10
711220    10
711402    10
711488    10
711751     4
Name: rating, Length: 2572, dtype: int64

In [13]:
np.average(reviews.query("movie_id == 816692").rating.value_counts().index)

5.0

In [17]:
reviw=reviews.query("user_id ==1").rating
reviw.head()

0    10
1    10
Name: rating, dtype: int64

In [20]:
max_date=reviews.groupby("movie_id").agg({'date':["max"]})
max_date

,date
,max
movie_id,
8,2014-04-08 18:20:11
10,2014-10-09 18:15:53
12,2015-08-10 23:16:19
25,2017-02-27 10:04:59
91,2013-11-23 18:59:55
...,...
8335880,2018-05-11 18:39:25
8342748,2018-05-26 18:50:01


In [1]:
##r=reviews.query("movie_id == 1454468").rating.value_counts()
#total_rating=r.values*r.index
#sum(total_rating)/r.values.sum()

avg_rating=[]
for i in range(id_length): 
    q="movie_id =="+str(id_length[i])
    r=reviews.query(q).rating.value_counts()
    total_rating=r.values*r.index
    
    rate=sum(total_rating)/r.values.sum()
    
    avg_rating.append(rate)
len(avg_rating)

In [35]:
id=movies['movie_id']
len(id)

31245

In [36]:
avg_rating=[]
for i in range(len(id)): 
    
    q="movie_id =="+str(id[i])
    r=reviews.query(q).rating.value_counts()
    
    total_rating=r.values*r.index
    
    rate=sum(total_rating)/r.values.sum()
    
    avg_rating.append(rate)
len(avg_rating)

31245

In [39]:
df=pd.DataFrame({'ID':id, 'AVG_rating': avg_rating})
df.head(10)

,ID,AVG_rating
0,8,5.000000
1,10,10.000000
2,12,10.000000
3,25,8.000000
4,91,6.000000
5,417,8.368421
6,439,6.750000
7,443,8.000000
8,628,4.500000
9,833,3.000000


In [41]:
df = df.sort_values(by=['AVG_rating'], ascending=False)
df

,ID,AVG_rating
7468,105107,10.0
19065,1684917,10.0
23819,2626338,10.0
16160,1149405,10.0
5402,85769,10.0
...,...,...
26581,3661976,1.0
17553,1421032,1.0
10929,259898,1.0
6188,93142,1.0


In [45]:
# df.to_csv(r"C:\Users\Vista\Desktop\rs\AVG_rating1.csv")

In [46]:
df2=pd.read_csv(r"C:\Users\Vista\Desktop\rs\AVG_rating1.csv")
len(df2)

31245

In [73]:
df_best=df2.query("AVG_rating > 5 and AVG_rating < 10 ")[['ID','AVG_rating']]
df_best.head()
len(df_best)

23796

In [74]:
df_best = df_best.sort_values(by=['AVG_rating'], ascending=False)
df_best

,ID,AVG_rating
1709,5512872,9.985836
1710,4148400,9.882353
1711,1629443,9.800000
1712,6798422,9.800000
1713,111341,9.800000
...,...,...
25500,2802136,5.051282
25501,4694544,5.041667
25502,1780762,5.040000
25503,1929308,5.037037


In [75]:
id=list(df_best['ID'].values)
rating=list(df_best['AVG_rating'].values)

In [76]:
ll=[]
l=[]
for i in range(0,10):
    q1="movie_id =="+str(id[i])
    d1=list(reviews.query(q1)['date'])
    d1.sort(reverse=True)
    
    q2="movie_id =="+str(id[i+1])
    d2=list(reviews.query(q2)['date'])
    d2.sort(reverse=True)

    if rating[i]>rating[i+1]:
        ll.append(id[i])
    elif rating[i]==rating[i+1]:
        if d1[0]>d2[0]:
            ll.append(id[i])
        else:
            l.append(id[i+1])
ll

[5512872, 4148400, 6798422, 111341, 423176, 5476944, 53114]

In [77]:
movie=[]
for i in range(0,len(ll)):
    q="movie_id =="+str(ll[i])
    name=movies.query(q)['movie']
    movie.append(name)
    
movie

[29679    Be Somebody (2016)
 Name: movie, dtype: object,
 27609    Birlesen Gonuller (2014)
 Name: movie, dtype: object,
 30864    Agnelli (2017)
 Name: movie, dtype: object,
 8066    Sátántangó (1994)
 Name: movie, dtype: object,
 13201    Shijie (2004)
 Name: movie, dtype: object,
 29641    Kuroshitsuji: Book of the Atlantic (2017)
 Name: movie, dtype: object,
 2185    Ningen no jôken (1959)
 Name: movie, dtype: object]

In [80]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
  
    #the movie that the user watched it
    q1="user_id =="+str(user_id)
    user_moive=list(reviews.query(q1)['movie_id'].values)
    l1=[]

    for i in range((n_top+len(user_moive))):
  
        q1="movie_id =="+str(id[i])
        d1=list(reviews.query(q1)['date'])
        d1.sort(reverse=True)
        
        q2="movie_id =="+str(id[i+1])
        d2=list(reviews.query(q2)['date'])
        d2.sort(reverse=True)
        

        if rating[i]>rating[i+1]:
            l1.append(id[i])
    
        elif rating[i]==rating[i+1]:
            if d1[0]>d2[0]:
                l1.append(id[i])
             
            else:
                l1.append(id[i+1])

        
        l2=[]
        for i in range(0,len(l1)):
            if l1[i] not in user_moive:
                l2.append(l1[i]) 
        
        top_movies=[]
        for i in range(len(l2)):
            q="movie_id =="+str(l2[i])
            name=movies.query(q)['movie']
            top_movies.append(name)
    
    return top_movies[0:n_top] # a list of the n_top movies as recommended

In [82]:
l=popular_recommendations(1, 20)
print(len(l),l)

20 [29679    Be Somebody (2016)
Name: movie, dtype: object, 27609    Birlesen Gonuller (2014)
Name: movie, dtype: object, 30864    Agnelli (2017)
Name: movie, dtype: object, 30864    Agnelli (2017)
Name: movie, dtype: object, 8066    Sátántangó (1994)
Name: movie, dtype: object, 13201    Shijie (2004)
Name: movie, dtype: object, 29641    Kuroshitsuji: Book of the Atlantic (2017)
Name: movie, dtype: object, 2715    Soy Cuba (1964)
Name: movie, dtype: object, 2185    Ningen no jôken (1959)
Name: movie, dtype: object, 2185    Ningen no jôken (1959)
Name: movie, dtype: object, 1926    Toute la mémoire du monde (1956)
Name: movie, dtype: object, 8672    When We Were Kings (1996)
Name: movie, dtype: object, 2735    Akahige (1965)
Name: movie, dtype: object, 22960    Crystal Lake Memories: The Complete History of...
Name: movie, dtype: object, 2449    Les dimanches de Ville d'Avray (1962)
Name: movie, dtype: object, 2449    Les dimanches de Ville d'Avray (1962)
Name: movie, dtype: object, 307

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [ ]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = # Your solution list here



In [ ]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy'])
```

In [87]:
q="genre in "+ str(['History', 'Comedy'])+ "and date in"+ str(['2015', '2016', '2017', '2018'])
m=movies.query(q)['movie_id']
len(m.values)

505

In [83]:
def popular_recs_filtered(user_id,n_top,years,genres):
    
    q="genre in "+ str(genres)+ "and date in"+ str(years)
    movie=movies.query(q)['movie_id']
    
    avg_rating=[]
    id=list(movie.values)
    
    for i in range(len(id)): 
        q="movie_id =="+str(id[i])
        r=reviews.query(q).rating.value_counts()
        total_rating=r.values*r.index
        rate=sum(total_rating)/r.values.sum()
        avg_rating.append(rate)


    df=pd.DataFrame({'ID':id, 'AVG_rating': avg_rating})
    df=df.sort_values(by=['AVG_rating'], ascending=False)
    df=df.query("AVG_rating > 5 and AVG_rating < 10 ")[['ID','AVG_rating']]
    rating=list(df['AVG_rating'])
    
    #the movie that the user watched it
    q1="user_id =="+str(user_id)
    user_moive=list(reviews.query(q1)['movie_id'].values)
    l1=[]

    for i in range((n_top+len(user_moive))):
        
        q1="movie_id =="+str(id[i])
        d1=list(reviews.query(q1)['date'])
        d1.sort(reverse=True)
        
        q2="movie_id =="+str(id[i+1])
        d2=list(reviews.query(q2)['date'])
        d2.sort(reverse=True)

        if rating[i]>rating[i+1]:
            l1.append(id[i])
    
        elif rating[i]==rating[i+1]:
            if d1[0]>d2[0]:
                l1.append(id[i])
             
            else:
                l1.append(id[i+1])

        
        l2=[]
        
        for i in range(0,len(l1)):
            if l1[i] not in user_moive and l1[i]  in id:
                l2.append(l1[i]) 
        
        top_movies=[]
        for i in range(len(l2)):
            q="movie_id =="+str(l2[i])
            name=movies.query(q)['movie']
            top_movies.append(name)
        
    
    return top_movies[0:n_top]  # a list of the n_top movies as recommended

In [84]:
top=popular_recs_filtered('1', 20, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy'])
print(len(top))
print(top)

20
[15053    The Wedding Ringer (2015)
Name: movie, dtype: object, 16839    The Last Film Festival (2016)
Name: movie, dtype: object, 16839    The Last Film Festival (2016)
Name: movie, dtype: object, 18117    Daddy's Home (2015)
Name: movie, dtype: object, 18117    Daddy's Home (2015)
Name: movie, dtype: object, 18582    Zoolander 2 (2016)
Name: movie, dtype: object, 19001    Entourage (2015)
Name: movie, dtype: object, 19001    Entourage (2015)
Name: movie, dtype: object, 19234    Office Christmas Party (2016)
Name: movie, dtype: object, 19777    Bad Santa 2 (2016)
Name: movie, dtype: object, 19777    Bad Santa 2 (2016)
Name: movie, dtype: object, 20157    Dirty Grandpa (2016)
Name: movie, dtype: object, 20157    Dirty Grandpa (2016)
Name: movie, dtype: object, 20556    Who Gets the Dog? (2016)
Name: movie, dtype: object, 20557    Drunk Wedding (2015)
Name: movie, dtype: object, 20622    Father Figures (2017)
Name: movie, dtype: object, 20629    The Comedian (2016)
Name: movie, dtype

In [85]:
top=popular_recs_filtered('100', 20, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy','Short|Horror'])
print(len(top))
print(top)

20
[15053    The Wedding Ringer (2015)
Name: movie, dtype: object, 16839    The Last Film Festival (2016)
Name: movie, dtype: object, 16839    The Last Film Festival (2016)
Name: movie, dtype: object, 18117    Daddy's Home (2015)
Name: movie, dtype: object, 18117    Daddy's Home (2015)
Name: movie, dtype: object, 18582    Zoolander 2 (2016)
Name: movie, dtype: object, 19001    Entourage (2015)
Name: movie, dtype: object, 19001    Entourage (2015)
Name: movie, dtype: object, 19234    Office Christmas Party (2016)
Name: movie, dtype: object, 19777    Bad Santa 2 (2016)
Name: movie, dtype: object, 19777    Bad Santa 2 (2016)
Name: movie, dtype: object, 20157    Dirty Grandpa (2016)
Name: movie, dtype: object, 20157    Dirty Grandpa (2016)
Name: movie, dtype: object, 20556    Who Gets the Dog? (2016)
Name: movie, dtype: object, 20557    Drunk Wedding (2015)
Name: movie, dtype: object, 20622    Father Figures (2017)
Name: movie, dtype: object, 20629    The Comedian (2016)
Name: movie, dtype